In [1]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def parse_row_as_element(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    # return Element(name, prevElemCombos)
    return {name : {'name': name, 'prevElemCombos': prevElemCombos, 'nextPossibleElements': []}}

def printx(listx):
    for x in listx:
        print(x)

def create_elements(csv_list):
    elements_arr = [parse_row_as_element(row) for row in csv_list]
    elements = elements_arr[0]
    for e_arr in elements_arr[1:]:
        elements.update(e_arr)
    return elements

In [2]:
filename = 'combinations_list_orig.csv'
csv_list = load_csv(filename)
elements = create_elements(csv_list)
# print(elements)

In [3]:
def create_tree(elements):
    elem_names = list(elements.keys())
    elements['BASE_ELEMENT'] = {'name': 'BASE_ELEMENTS', 'prevElemCombos': [], 'nextPossibleElements': []}
    for elem in elem_names:
        for prevElemCombo in elements[elem]['prevElemCombos']:
            for prevElem in prevElemCombo:
                try:
                    elements[prevElem]['nextPossibleElements'].append(elem)
                except KeyError:
                    elements[prevElem] = {'name': prevElem, 'prevElemCombos': [], 'nextPossibleElements': [elem]}         

    return elements 

elements = create_tree(elements)

In [4]:
from functools import lru_cache
cache = lru_cache(maxsize=None)

# @cache

def get_element_score(element, elements, level=0):
    if level == 0:
        return len(elements[element]['nextPossibleElements'])
    else:
        return sum([get_element_score(elem, elements, level-1) for elem in elements[element]['nextPossibleElements']])

# @cache
def get_valid_combination_list(elements, strs=False):
    arr = []
    for elem in elements:
        for c, combo in enumerate(elements[elem]['prevElemCombos']):
            arr.append('+'.join(combo + [elem]))
    arr = list(set(arr))
    return arr if strs else [a.split('+') for a in arr] 
    
    

In [9]:
from joblib import Parallel, delayed

def get_combos(current_elements, elements):
    valid_combos = get_valid_combination_list(elements)
    test_combos = ['+'.join([a, b]) for a in current_elements for b in current_elements]
    print(test_combos)
    test_combos = list(set(test_combos))
    test_combos = [a for a in test_combos if a in valid_combos]
    return test_combos


def get_next_best_element(current_elements, elements):
    valid_combos = get_combos(current_elements, elements)
    all_next_elements = [arr[-1] for arr in valid_combos]
    valid_combos = ['+'.join(a[:-1]) for a in valid_combos]
    all_next_elements = list(set(all_next_elements))
    get_rankings = []
    get_rankings = Parallel(n_jobs=-1)(delayed(get_element_score)(elem, elements) for elem in all_next_elements)
    all_next_elements = [[all_next_elements[i], get_rankings[i]] for i in range(len(all_next_elements))]
    all_next_elements = sorted(all_next_elements, key=lambda x: x[1], reverse=True)
    return all_next_elements

In [6]:
base_elements = elements['BASE_ELEMENT']['nextPossibleElements']

current_elements = base_elements

get_next_best_element(current_elements, elements)

[]

In [8]:
get_valid_combination_list(elements)

[['sickness', 'leaf', 'herb'],
 ['time', 'electricity', 'clock'],
 ['house', 'doctor', 'hospital'],
 ['rain', 'smoke', 'acid rain'],
 ['day', 'vampire', 'dust'],
 ['bird', 'snow', 'penguin'],
 ['sailor', 'pirate ship', 'pirate'],
 ['snow', 'wagon', 'sledge'],
 ['house', 'snow', 'igloo'],
 ['light', 'ocean', 'lighthouse'],
 ['ice', 'cloud', 'hail'],
 ['river', 'metal', 'bridge'],
 ['rainbow', 'rainbow', 'double rainbow!'],
 ['livestock', 'mountain', 'goat'],
 ['tool', 'light', 'flashlight'],
 ['mud', 'sun', 'brick'],
 ['castle', 'night', 'ghost'],
 ['moon', 'butterfly', 'moth'],
 ['plant', 'sun', 'sunflower'],
 ['tree', 'wind', 'leaf'],
 ['dough', 'fruit', 'pie'],
 ['sea', 'antarctica', 'iceberg'],
 ['axe', 'electricity', 'chainsaw'],
 ['metal', 'sun', 'gold'],
 ['corpse', 'time', 'bone'],
 ['wheat', 'stone', 'flour'],
 ['egg', 'hay', 'nest'],
 ['corpse', 'life', 'zombie'],
 ['paper', 'pencil', 'letter'],
 ['ring', 'planet', 'saturn'],
 ['human', 'medusa', 'statue'],
 ['star', 'star', '